In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
import sklearn
import pandas as pd 
import matplotlib.pyplot as plt
from string import digits
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# from keras.utils import to_categorical
import keras as ks


from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from sklearn.mixture import GaussianMixture

import seaborn as sns
#sns.set_theme()

np.set_printoptions(precision=3, suppress=True)

In [9]:
from matplotlib.colors import LogNorm



def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=35, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=2, linewidths=12,
                color=cross_color, zorder=11, alpha=1)

def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z,
                linewidths=2, colors='r', linestyles='dashed')
    
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)

    plt.xlabel("Principle Component 1", fontsize=14)
    if show_ylabels:
        plt.ylabel("Principle Component 2", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)
        
        
def import_data():
    training_data = pd.read_csv("ref_aug_pre.csv")
    product_names = unique_product_names(training_data)

    def unique_product_names(training_data):
        product_names = training_data['Produkt'].unique()
        return product_names
    
    def find_multiple_class(product_array):
        
        def has_numbers(inputString):
            return any(char.isdigit() for char in inputString)
        
        class_name_with_index = []
        for i in range(0,len(product_array)):
            if has_numbers(product_array[i]) == True:
                class_name_with_index.append(product_array[i])
        return class_name_with_index

    class_name_with_indices = find_multiple_class(product_names)

    array = []
    for i in range(0, len(class_name_with_indices)):
        array.append(class_name_with_indices[i].translate({ord(k): None for k in digits}))
    unique_names = list(set(array))  

    sorted_class_array = []
    for i in range(0, len(unique_names)):
        sorted_class_array.append([])
        for j in range(0, len(array)):
            if unique_names[i] == array[j] and unique_names[i] in class_name_with_indices[j]:
                sorted_class_array[i].append(class_name_with_indices[j])
    return [training_data, unique_names, class_name_with_indices, sorted_class_array]

import_data()
        

# pca = PCA(n_components=2)
# principalComponents = pca.fit_transform(transformed_training_data)
# print("Variances on each principal component: {}".format(pca.explained_variance_ratio_))

# principalDf = pd.DataFrame(data = principalComponents
#              , columns = ['principal component 1', 'principal component 2'])
# '''Add back the colum with the class names'''
# finalDf = pd.concat([principalDf, training_data[['Produkt']]], axis = 1)
    


def GaussianMixtureModel():
    length_class_array = np.shape(class_array)[0]
    for i in range(length_class_array):
        target_array = class_array[i][:]
        df_final = finalDf[finalDf['Produkt'].isin(target_array)]

        '''prepare for clustering'''
        df_final_with = df_final.copy()
        df_final.drop("Produkt",inplace = True, axis =1)
        np_final_array = df_final.to_numpy()
        
        
        pca = PCA(n_components=2)
        principalComponents = pca.fit_transform(transformed_training_data)
        print("Variances on each principal component: {}".format(pca.explained_variance_ratio_))

        principalDf = pd.DataFrame(data = principalComponents
                    , columns = ['principal component 1', 'principal component 2'])
        '''Add back the colum with the class names'''
        finalDf = pd.concat([principalDf, training_data[['Produkt']]], axis = 1)
        
        
        def pca_for_class(array):
            for i in range(0, np.shape(array)[0]):
                fig = plt.figure(figsize = (8,8))
                ax = fig.add_subplot(1,1,1) 
                ax.set_xlabel('Principal Component 1', fontsize = 15)
                ax.set_ylabel('Principal Component 2', fontsize = 15)
                ax.set_title('PCA Decomposition', fontsize = 15)
                target_array = array[i][:]
                for target in target_array:
                    indicesToKeep = finalDf['Produkt'] == target
                    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1'], finalDf.loc[indicesToKeep, 'principal component 2'], s = 20)
                    ax.legend(target_array)
                ax.grid(True)
                plt.show()
            pca_for_class(class_array)
                
    #target_array = class_array[25][:]
    
        number_array = []
        '''Make sure all the numbering starts at 1 of the class names'''
        for i in df_final_with["Produkt"]:
            number = [int(j) for j in i.split() if j.isdigit()]
            if len(number) == 1:
                number_array.append(number[0]-1)
            else:
                combined_number = int("{}{}".format(number[0],number[1])) 
                number_array.append(combined_number-1)
        df_final_with["cluster"] = number_array
        
        
        gmm = GaussianMixture(n_components = len(target_array) ,covariance_type = 'full', random_state = 0, n_init = 10).fit(df_final[["principal component 1", "principal component 2"]])
        sns.scatterplot(data= df_final, x = "principal component 1", y = "principal component 2") 
        print("Did the algorithm converge?: {}".format(gmm.converged_))
        gmm.means_
        labels = gmm.predict(df_final[["principal component 1", "principal component 2"]])
        
        correct_array = []
        for i in range(len(labels)):
            if df_final_with.iloc[i].at["cluster"] == labels[i]:
                correct_array.append(True)
            else:
                correct_array.append(False)
        df_final_with["Status"] = correct_array
        count = df_final_with['Status'].sum()
        count_totalnr = len(df_final_with)
        accuracy = count/count_totalnr
        print(accuracy)
        plot_data = df_final_with[['principal component 1','principal component 2']].to_numpy()
        plt.figure(figsize=(8, 4))
        plot_gaussian_mixture(gmm, plot_data)
        plt.show()

        

FileNotFoundError: [Errno 2] No such file or directory: 'ref_aug_pre.csv'